# Linear Regression: From Intuition to Sklearn

---

## Learning Objectives

By the end of this notebook, you will be able to:

- Understand the intuition behind the line of best fit and residuals
- Derive and apply the least squares cost function
- Implement simple linear regression from scratch using NumPy
- Use `sklearn.linear_model.LinearRegression` for single and multi-feature regression
- Visualize regression results: fit lines, residuals, and actual vs predicted plots
- Apply best practices: train/test splits and pipelines with scaling

## Prerequisites

- Basic Python (NumPy, Matplotlib)
- Understanding of mean, variance, and basic algebra
- Familiarity with train/test splitting concepts

## Table of Contents

1. [The Line of Best Fit: Intuition](#1-the-line-of-best-fit-intuition)
2. [Residuals and the Least Squares Cost Function](#2-residuals-and-the-least-squares-cost-function)
3. [The Normal Equation](#3-the-normal-equation)
4. [Linear Regression from Scratch (NumPy)](#4-linear-regression-from-scratch-numpy)
5. [Linear Regression with Sklearn](#5-linear-regression-with-sklearn)
6. [Multi-Feature Regression](#6-multi-feature-regression)
7. [Visualizations](#7-visualizations)
8. [Best Practices](#8-best-practices)
9. [Common Mistakes](#9-common-mistakes)
10. [Exercise](#10-exercise)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.datasets import fetch_california_housing

np.random.seed(42)
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

---

## 1. The Line of Best Fit: Intuition

Linear regression finds the **straight line** (or hyperplane) that best describes the relationship between input features $x$ and a continuous target $y$.

**The model:**

$$\hat{y} = w_0 + w_1 x_1 + w_2 x_2 + \dots + w_p x_p$$

Where:
- $w_0$ is the **intercept** (bias term)
- $w_1, \dots, w_p$ are the **coefficients** (weights) for each feature
- $\hat{y}$ is the **predicted** value

For a single feature, this simplifies to the familiar equation of a line: $\hat{y} = w_0 + w_1 x$.

In [ ]:
# Generate simple 1D data
np.random.seed(42)
X_simple = 2 * np.random.rand(50, 1)
y_simple = 3 + 4 * X_simple.flatten() + np.random.randn(50) * 0.8

plt.scatter(X_simple, y_simple, alpha=0.7, edgecolors="k", label="Data points")
plt.xlabel("x")
plt.ylabel("y")
plt.title("A Simple Dataset — Where Should the Line Go?")
plt.legend()
plt.tight_layout()
plt.show()

---

## 2. Residuals and the Least Squares Cost Function

A **residual** is the difference between the observed value and the predicted value:

$$e_i = y_i - \hat{y}_i$$

The **Mean Squared Error (MSE)** cost function sums the squared residuals:

$$J = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$$

**Why squared?**
- Penalizes large errors more than small ones
- Makes the function differentiable (smooth optimization)
- Positive and negative errors don't cancel out

The goal of linear regression is to find $w_0, w_1, \dots, w_p$ that **minimize** $J$.

In [ ]:
# Visualize residuals for two candidate lines
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bad fit
w0_bad, w1_bad = 1.0, 2.0
y_pred_bad = w0_bad + w1_bad * X_simple.flatten()
axes[0].scatter(X_simple, y_simple, alpha=0.7, edgecolors="k")
axes[0].plot(X_simple, y_pred_bad, "r-", linewidth=2, label=f"y = {w0_bad} + {w1_bad}x")
for i in range(len(X_simple)):
    axes[0].plot([X_simple[i, 0], X_simple[i, 0]], [y_simple[i], y_pred_bad[i]],
                 "r--", alpha=0.3, linewidth=0.8)
mse_bad = np.mean((y_simple - y_pred_bad) ** 2)
axes[0].set_title(f"Poor Fit — MSE = {mse_bad:.2f}")
axes[0].legend()
axes[0].set_xlabel("x")
axes[0].set_ylabel("y")

# Good fit (close to true values)
w0_good, w1_good = 3.0, 4.0
y_pred_good = w0_good + w1_good * X_simple.flatten()
axes[1].scatter(X_simple, y_simple, alpha=0.7, edgecolors="k")
axes[1].plot(X_simple, y_pred_good, "g-", linewidth=2, label=f"y = {w0_good} + {w1_good}x")
for i in range(len(X_simple)):
    axes[1].plot([X_simple[i, 0], X_simple[i, 0]], [y_simple[i], y_pred_good[i]],
                 "g--", alpha=0.3, linewidth=0.8)
mse_good = np.mean((y_simple - y_pred_good) ** 2)
axes[1].set_title(f"Better Fit — MSE = {mse_good:.2f}")
axes[1].legend()
axes[1].set_xlabel("x")
axes[1].set_ylabel("y")

plt.tight_layout()
plt.show()

---

## 3. The Normal Equation

For ordinary least squares, there is a **closed-form solution** called the Normal Equation:

$$\mathbf{w} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

Where $\mathbf{X}$ includes a column of ones for the intercept.

**Key points:**
- Directly computes the optimal weights in one step (no iteration)
- Computational complexity is $O(n \cdot p^2 + p^3)$ — expensive for large $p$
- Requires $\mathbf{X}^T\mathbf{X}$ to be invertible (fails with perfect multicollinearity)
- Sklearn uses optimized solvers (SVD-based) that are more numerically stable

In [ ]:
# Normal equation implementation
X_b = np.c_[np.ones((X_simple.shape[0], 1)), X_simple]  # add bias column
w_normal = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y_simple

print(f"Normal Equation solution:")
print(f"  Intercept (w0): {w_normal[0]:.4f}")
print(f"  Slope     (w1): {w_normal[1]:.4f}")
print(f"  (True values: w0=3.0, w1=4.0)")

---

## 4. Linear Regression from Scratch (NumPy)

Let's implement simple linear regression from scratch using both the normal equation and a manual gradient descent approach.

In [ ]:
class SimpleLinearRegression:
    """Linear regression using the Normal Equation."""

    def __init__(self):
        self.weights = None

    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        self.weights = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
        return self

    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b @ self.weights

    @property
    def intercept_(self):
        return self.weights[0]

    @property
    def coef_(self):
        return self.weights[1:]


# Fit our scratch model
model_scratch = SimpleLinearRegression()
model_scratch.fit(X_simple, y_simple)

print(f"From-scratch model:")
print(f"  Intercept: {model_scratch.intercept_:.4f}")
print(f"  Coefficient: {model_scratch.coef_[0]:.4f}")

In [ ]:
# Visualize the from-scratch fit with residuals
y_pred_scratch = model_scratch.predict(X_simple)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter + fit line
X_sorted = np.sort(X_simple, axis=0)
y_line = model_scratch.predict(X_sorted)
axes[0].scatter(X_simple, y_simple, alpha=0.7, edgecolors="k", label="Data")
axes[0].plot(X_sorted, y_line, "r-", linewidth=2, label="Fit line")
axes[0].set_xlabel("x")
axes[0].set_ylabel("y")
axes[0].set_title("From-Scratch Linear Regression")
axes[0].legend()

# Residuals
residuals = y_simple - y_pred_scratch
axes[1].scatter(y_pred_scratch, residuals, alpha=0.7, edgecolors="k")
axes[1].axhline(y=0, color="r", linestyle="--", linewidth=1)
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("Residuals")
axes[1].set_title("Residual Plot")

plt.tight_layout()
plt.show()

mse_scratch = np.mean(residuals ** 2)
print(f"MSE: {mse_scratch:.4f}")

---

## 5. Linear Regression with Sklearn

Sklearn's `LinearRegression` provides a production-ready implementation with a clean API.

In [ ]:
# Sklearn LinearRegression on the same data
model_sk = LinearRegression()
model_sk.fit(X_simple, y_simple)

print("Sklearn LinearRegression:")
print(f"  intercept_:      {model_sk.intercept_:.4f}")
print(f"  coef_:           {model_sk.coef_}")
print(f"  n_features_in_:  {model_sk.n_features_in_}")

# Predict
y_pred_sk = model_sk.predict(X_simple)

# Compare with our scratch model
print(f"\nMax difference between scratch and sklearn predictions: "
      f"{np.max(np.abs(y_pred_scratch - y_pred_sk)):.2e}")

---

## 6. Multi-Feature Regression

Linear regression extends naturally to multiple features.

In [ ]:
# Generate multi-feature synthetic data
np.random.seed(42)
n_samples = 200
n_features = 5

X_multi = np.random.randn(n_samples, n_features)
true_weights = np.array([3.0, -1.5, 0.0, 2.0, -0.5])
true_intercept = 5.0
y_multi = X_multi @ true_weights + true_intercept + np.random.randn(n_samples) * 0.5

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42
)

# Fit model
model_multi = LinearRegression()
model_multi.fit(X_train, y_train)

print("Multi-feature regression results:")
print(f"  Intercept: {model_multi.intercept_:.4f} (true: {true_intercept})")
print(f"  Coefficients:")
for i, (learned, true) in enumerate(zip(model_multi.coef_, true_weights)):
    print(f"    w{i+1}: {learned:+.4f}  (true: {true:+.1f})")
print(f"  n_features_in_: {model_multi.n_features_in_}")

# Evaluate
y_pred_train = model_multi.predict(X_train)
y_pred_test = model_multi.predict(X_test)

print(f"\n  Train R2: {r2_score(y_train, y_pred_train):.4f}")
print(f"  Test  R2: {r2_score(y_test, y_pred_test):.4f}")

---

## 7. Visualizations

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Actual vs Predicted (test set)
axes[0].scatter(y_test, y_pred_test, alpha=0.7, edgecolors="k")
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
axes[0].plot([min_val, max_val], [min_val, max_val], "r--", linewidth=2, label="Perfect prediction")
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predicted")
axes[0].set_title("Actual vs Predicted (Test Set)")
axes[0].legend()

# Coefficient comparison
feature_names = [f"w{i+1}" for i in range(n_features)]
x_pos = np.arange(n_features)
width = 0.35
axes[1].bar(x_pos - width/2, true_weights, width, label="True", color="steelblue", alpha=0.8)
axes[1].bar(x_pos + width/2, model_multi.coef_, width, label="Learned", color="coral", alpha=0.8)
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(feature_names)
axes[1].set_ylabel("Weight value")
axes[1].set_title("True vs Learned Coefficients")
axes[1].legend()
axes[1].axhline(y=0, color="gray", linestyle="-", linewidth=0.5)

plt.tight_layout()
plt.show()

---

## 8. Best Practices

- **Always split your data** into train and test sets before fitting
- **Use a Pipeline** to combine preprocessing and model fitting
- **Scale features** when they have different units or magnitudes
- **Check residual plots** to validate model assumptions
- **Report metrics on the test set**, not the training set

In [ ]:
# Best practice: Pipeline with StandardScaler
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", LinearRegression())
])

pipeline.fit(X_train, y_train)
y_pred_pipe = pipeline.predict(X_test)

print(f"Pipeline Test R2: {r2_score(y_test, y_pred_pipe):.4f}")
print(f"Pipeline Test MSE: {mean_squared_error(y_test, y_pred_pipe):.4f}")
print("\nNote: For linear regression with OLS, scaling does not change R2,")
print("but it IS essential for regularized models (Ridge, Lasso).")

---

## 9. Common Mistakes

| Mistake | Why It's a Problem | Fix |
|---|---|---|
| Not splitting data | Overly optimistic metrics | Use `train_test_split` |
| Ignoring residual plots | Miss assumption violations | Always plot residuals |
| Using R2 alone | R2 can be misleading (e.g., nonlinear data) | Check MSE + residuals |
| Forgetting to scale | Coefficients not comparable, hurts regularized models | Use `StandardScaler` in a `Pipeline` |
| Fitting on test data | Data leakage | Only `transform` on test, never `fit` |
| Extrapolating beyond training range | Linear models assume the trend continues | Be cautious about prediction range |

---

## 10. Exercise

**Task:** Fit a linear regression model on the **California Housing** dataset and report the R2 score.

Steps:
1. Load the data with `fetch_california_housing()`
2. Split into train (80%) and test (20%) with `random_state=42`
3. Build a Pipeline: `StandardScaler` + `LinearRegression`
4. Fit on training data, predict on test data
5. Report R2 and create an actual vs predicted plot

In [ ]:
# --- Exercise Solution ---

# Step 1: Load data
housing = fetch_california_housing()
X_housing = housing.data
y_housing = housing.target
feature_names_housing = housing.feature_names

print(f"Dataset shape: {X_housing.shape}")
print(f"Features: {feature_names_housing}")
print(f"Target: Median house value (in $100,000s)")

# Step 2: Split
X_h_train, X_h_test, y_h_train, y_h_test = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=42
)

# Step 3: Pipeline
housing_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", LinearRegression())
])

# Step 4: Fit and predict
housing_pipeline.fit(X_h_train, y_h_train)
y_h_pred = housing_pipeline.predict(X_h_test)

# Step 5: Report
r2 = r2_score(y_h_test, y_h_pred)
rmse = np.sqrt(mean_squared_error(y_h_test, y_h_pred))
print(f"\nTest R2:   {r2:.4f}")
print(f"Test RMSE: {rmse:.4f}")

# Actual vs Predicted
plt.figure(figsize=(7, 5))
plt.scatter(y_h_test, y_h_pred, alpha=0.3, s=10, edgecolors="k", linewidths=0.3)
mn = min(y_h_test.min(), y_h_pred.min())
mx = max(y_h_test.max(), y_h_pred.max())
plt.plot([mn, mx], [mn, mx], "r--", linewidth=2, label="Perfect prediction")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title(f"California Housing — Actual vs Predicted (R2={r2:.3f})")
plt.legend()
plt.tight_layout()
plt.show()